# Исследование надежности заемщиков

## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание данных

`children` — количество детей в семье

`days_employed` — общий трудовой стаж в днях

`dob_years` — возраст клиента в годах

`education` — уровень образования клиента

`education_id` — идентификатор уровня образования

`family_status` — семейное положение

`family_status_id` — идентификатор семейного положения

`gender` — пол клиента

`income_type` — тип занятости

`debt` — имел ли задолженность по возврату кредитов

`total_income` — ежемесячный доход

`purpose` — цель получения кредита



**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:** `/Users/fen1x/Downloads/data.csv`

In [90]:
# импортируйте библиотеку pandas
import pandas as pd

# прочитайте csv-файл
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('/Users/fen1x/Downloads/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**


In [91]:
#
print(data.head(20))



    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**



In [92]:
#
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**



In [93]:
#
print(data.isna().sum())



children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**



In [94]:
#
#Группируем данные по столбцу 'income_type' и высчитываем медианное значение данных о доходах(столбец `total_income`)
#медианное значение доходов (столбец `total_income`) по каждой группе
income_median = data.groupby('income_type')['total_income'].transform('median')
# Заменяем пропуски медианным значением 
data['total_income'] = data['total_income'].fillna(income_median)
# Проверим наличие пропущенных значений для столбца total_income
data['total_income'].isna().sum()

0

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**



In [95]:
#
data['days_employed'] = data['days_employed'].abs()



**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**



In [96]:
#
print(data.groupby('income_type')['days_employed'].median())



income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**



In [97]:
#
print(data['children'].unique())



[ 1  0  3  2 -1  4 20  5]


**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**



In [98]:
#
data = data.loc[(data['children'] != 20)&(data['children'] != -1)]



**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**



In [99]:
#
print(data['children'].unique())



[1 0 3 2 4 5]


### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**



In [100]:
# Группируем данные по типу занятости(income_type) и расчитываем медианное значение трудового стажа для каждого типа
days_employed_median = data.groupby('income_type')['days_employed'].transform('median')
# Заполняем пропуски в столбце days_employed медианными значениями по каждому типу занятости
data['days_employed'] = data['days_employed'].fillna(days_employed_median) 


**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**


In [101]:
print(data.isna().sum())



children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**



In [102]:
#
data['total_income'] = data['total_income'].astype('int') 



### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**



In [103]:
#
data['education'] = data['education'].str.lower()



**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**



In [104]:
# посчитайте дубликаты
print(data.duplicated().sum())



71


In [105]:
# удалите дубликаты
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**



In [106]:
try:    
    # создайте функцию categorize_income()
    def categorize_income(row):
        income = row['total_income']
        if income <= 30000:
            return "E"
        if income <= 50000:
            return "D"
        if income <= 200000:
            return "C"
        if income <= 1000000:
            return "B"
        if income > 1000000:
            return "A"
except:
    print('возникла ошибка') 

In [107]:
# примените функцию методом apply()
data['total_income_category'] = data.apply(categorize_income, axis = 1)

In [108]:
# Проверим какие категории находятся в столбце total_income_category
data['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**



In [109]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**



In [110]:
# создайте функцию categorize_purpose()
def categorize_purpose(row):
    purpose = row['purpose']
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose or 'недв' in purpose:
        return 'операции с недвижимостью'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'образов' in purpose:
        return 'получение образования'
    else:
        return 'категория не присвоена'
    

In [111]:
# примените функцию методом apply()
data['purpose_category'] = data.apply(categorize_purpose, axis = 1)

In [112]:
# Проверим перечень уникальных категорий в столбце purpose_category, определим всем ли строкам присвоена категория
print(data['purpose_category'].unique())

['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


**Так как значение ``'категория не присвоена'`` отсутствует, можно сделать вывод, что карегории были присвоены всем данным из столбца ``purpose``**

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

**Определим сколько должников имеет детей, а у какого количества их нет и посчитаем процент должников от общего числа заемщиков. Для наглядности даных сформируем сводную таблицу**

In [113]:
# Создадим столбец children_category с категориями семей: "бездетные семьи" и "семьи с детьми",
# используя функцию categorize_children

def categorize_children(row):
    if row['children'] == 0:
        return "бездетные семьи"
    if row['children'] == 1:
        return "семьи с одним ребенком"
    if row['children'] == 2:
        return "семьи с двумя детьми"
    if row['children'] > 2:
        return "многодетные семьи"    
    
    
data['children_category'] = data.apply(categorize_children, axis=1)


# Создадим столбец debt_category с категориями семей: "Семьи без задолженности" и "Семьи с задолженностью",
# с помощью функции categorize_debt
def categorize_debt(row):
    if row['debt'] == 0:
        return 'Заёмщики без задолженности'
    if row['debt'] != 0:
        return 'Заёмщики с задолженностью'
    
data['debt_category'] = data.apply(categorize_debt, axis=1)

# Представим данные в сводной таблице
# Создадим функцию, которая будет создавать сводную таблицу по заданной категории
# и производить дополнительные расчеты в сводной таблице

def df_pivot(categori):
# Создадим сводную таблицу   
    df = data.pivot_table(index= categori, 
                              columns='debt_category',
                              values='debt', 
                              aggfunc='count')
# Рассчитаем общее количество заемщиков
    df['Всего'] = df['Заёмщики без задолженности'] +    \
                              df['Заёмщики с задолженностью']
    
# Рассчитаем процент должников от общего количества заемщиков, округлив число до целых и поменяв его тип на целые числа     
    df['% должников от общего числа'] =(round((df['Заёмщики с задолженностью'] /   \
                                                   df['Всего']* 100),2))
# Возвращаем сводную таблицу, с сортировкой по убыванию % должников от общего числа заемщиков    
    return df.sort_values('% должников от общего числа',ascending=False)

In [114]:
# Строим сводную таблицу, с разбивкой по категориям столбца children_category
df_pivot('children_category')

debt_category,Заёмщики без задолженности,Заёмщики с задолженностью,Всего,% должников от общего числа
children_category,,,,
семьи с двумя детьми,1858,194,2052,9.45
семьи с одним ребенком,4364,444,4808,9.23
многодетные семьи,349,31,380,8.16
бездетные семьи,13028,1063,14091,7.54


**Вывод:**

 **Процент семей-должников *`с одним или двумя детьми`* незначительно выше, чем семей-должников *`без детей`* и *`многодетных семей`*. При этом следует отметить, что семьи с детьми берут кредиты реже в 2 раза**

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [115]:
# Посмотрим какие уникальные значения имеет столбец family_status
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [116]:
# Построим сводную таблицу с разбивкой заемщиков по семейному положению и учету отсутствия или наличия задолженности,
# используя функцию df_pivot
df_pivot('family_status')

debt_category,Заёмщики без задолженности,Заёмщики с задолженностью,Всего,% должников от общего числа
family_status,,,,
Не женат / не замужем,2523,273,2796,9.76
гражданский брак,3749,385,4134,9.31
женат / замужем,11334,927,12261,7.56
в разводе,1105,84,1189,7.06
вдовец / вдова,888,63,951,6.62


**Вывод:** 

**Таблица наглядно показывает, что зависимость между семейным положением и возвратом кредита в срок имеется. Самый большой процент невозврата - *`у неженатых и незамужних людей (10%)`*. При этом, заемщики, находящиеся в *`официальном браке`*, берут кредиты значительно чаще одиноких людей.**

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [117]:
# Построим сводную таблицу с разбивкой по уровню дохода и учету отсутствия или наличия задолженности
df_pivot('total_income_category')


debt_category,Заёмщики без задолженности,Заёмщики с задолженностью,Всего,% должников от общего числа
total_income_category,,,,
E,20,2,22,9.09
C,14568,1353,15921,8.50
A,23,2,25,8.00
B,4660,354,5014,7.06
D,328,21,349,6.02


**Вывод:** 

**Ожидаемо, самый большой процент *(9%)* невозврата оказался у заёмщиков с доходом категории *`E, (доход до 30 000)`*, но эта группа является малочисленной, как и группа заёмщиков с самым высоким доходом *`(А, доход от 1 000 000 и выше)`* и соответственно не может отражать общую тенденцию**

**Если сравнивать две наиболее многочисленные группы - *`С, с доходом от 50 001 до 200 000`* и *`В, с доходом от 200 001 до 1 000 000`*, то можно заметить, что заемщики, имеющие более высокий уровень дохода *`(категория В)`*, чаще возвращают кредиты в срок.**

#### 3.4 Как разные цели кредита влияют на его возврат в срок?



In [118]:
# Построим сводную таблицу с разбивкой по целям кредита и учету отсутствия или наличия задолженности
df_pivot('purpose_category')

debt_category,Заёмщики без задолженности,Заёмщики с задолженностью,Всего,% должников от общего числа
purpose_category,,,,
операции с автомобилем,3879,400,4279,9.35
получение образования,3619,369,3988,9.25
проведение свадьбы,2130,183,2313,7.91
операции с недвижимостью,9971,780,10751,7.26


**Вывод:** 

**Чаще всего кредиты берутся для *`операций с недвижимостью`* и при этом уровень невозврата займов с такой целью самый низкий *`(7.26%)`*** 
**По кредитам на *`операции с автомобилем`* и *`получение образования`* наиболее высокий уровень невозврата займов *`(9.35 и 9.25 соответственно%)`***

**Следует отметить, что, как и в предыдущих сравнениях задолженников различных категорий, разница между процентаими невозврата кредитов низкая.**


#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 
Так как имелось равное количество пропусков в двух столбцах `days_employed` и `total_income`, можно предположить что произошел сбой при внеесении в базу или выгрузке данных


#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Так как в данных столбцов `days_employed` и `total_income` встречались аномально большие значения, выборка  не была равномерной, то наиболее подходящим для заполнения было именно *медианное значение*.


### Шаг 4: общий вывод.

**В ходе работы требовалось разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Были получены входные данные от банка — статистика о платёжеспособности клиентов.**

**Перед анализом данных и проверкой гипотез была проведена предобработка данных.
Предобработка обнаружила следуюшие проблемы в данных:**
- Пропуски в данных столбцов `days_employed` и `total_income`, которые в ходе предобработки были заполнены медианными значениями. Отсутствие информации возможно было вызвано ошибкой при выгрузке данных, или по какой-то причине не была внесена в базу данных;
- Были изменены отрицательные значения в столбце `days_employed` с помощью модуля;
- Обработаны неявные дубликаты в столбце `education` с помощью приведения данных к нижнему регистру;
- Заменен вещественный тип данных в столбце `total_income` на целочисленный;
- Были удалены дубликаты строк
- Была произведена категоризация данных о доходах заёмщиков и их целях взятия кредитов 

**При анализе данных по кредитам были сделаны следующие выводы:**
 - Было определено что процент невозврата кредитов `семей-должников с детьми` выше, чем `семей-должников без детей`. При этом, разница между ними незначительна. Так же следует отметить, что `семьи с детьми` берут кредиты реже в 2 раза, чем `семьи без детей`.
 - Была найдена зависимость между семейным положением и возвратом кредита в срок. Самый большой процент невозврата - у `неженатых и незамужних людей (10%)`. При этом, заемщики, находящиеся в официальном браке, берут кредиты значительно чаще одиноких людей.
 - Так же подтвердилась гипотеза о зависимости дохода заёмщиков и возврате ими кредита в срок. Заемщики, имеющие более высокий уровень дохода *`(категория В с доходом от 200 001 до 1 000 000)`*, чаще возвращают кредиты в срок. 
 - Различные цели кредита так же влияют на возврат его в срок. Чаще всего кредиты берутся для `операций с недвижимостью` и при этом уровень невозврата займов с такой целью самый низкий (7.26%) По кредитам на `операции с автомобилем` и `получение образования` наиболее высокий уровень невозврата займов (9.35 и 9.25 соответственно%)
 
 Так как первоначальные данные имели ряд несовершенств, можно порекоммендовать банку проверить технические проблемы, все ли данные попадают в выгрузку, и сделать обязательные поля для заполнения, чтобы исключить человеческий фактор. Поскольку в данных о количестве отработанных дней имелись отрицательные значения, можно технически ограничить ввод данных исключительно неотрицательными целыми числами. А для исключения неявных повторов в целях взятия кредита, можно предусмотреть выбор из готового списка целей.